In [ ]:
## NODES

In [1]:
import os
# from pathlib import Path

# Load environment variables
from dotenv import load_dotenv

load_dotenv(override=True)
if not os.environ.get("OPENAI_API_KEY"):
    print("Error: OPENAI_API_KEY not found")
    exit(1)

# Import simple config and loader
from configs.config_paths import ConfigPaths
from utils.data_loader import load_data

# Import the graph builder

# Import RAG config
from rag.config_rag import RAGConfig

from configs.config_datasets import DatasetColumnMappings
from utils.data_loader import validate_expected_columns_in_masters

from configs.config_agent import ConfigAgents


"""Main execution flow - simple and clean."""

print("=" * 60)
print("BUSINESS GLOSSARY FILLING - AGENTIC WORKFLOW")
print("=" * 60)

# 1. Setup configuration
config_paths = ConfigPaths()
rag_default_config = RAGConfig(project_root=config_paths.project_root)
config_datasets = DatasetColumnMappings()
config_agentic = ConfigAgents()

print(f"\nProject root: {config_paths.project_root}")
print(f"Embedding model: {rag_default_config.embedding_model}")
print(f"LLM model: {config_agentic.llm_model}\n")

# 2. Load data

sample_dict, bg_dict, ds_dict = load_data(config_paths, config_datasets)
print("\nMake sure these files exist:")
print(f"  - {config_paths.main_dataset}")
print(f"  - {config_paths.master_glossary}")
print(f"  - {config_paths.data_stewards}")



# 3. Validate if files contains columns which are expected
bg_masters_columns = config_datasets.column_mappings_master_bg
ds_master_columns = config_datasets.column_mappings_master_data_owners
validate_expected_columns_in_masters(bg_dict, bg_masters_columns)
validate_expected_columns_in_masters(ds_dict, ds_master_columns)




# 3. Setup initial state
# initial_state = {
#     "framework_def": config_datasets.get_framework_dict(),
#     "source_original_table": sample_dict,
#     "master_business_glossary": bg_dict,
#     "master_data_owner": ds_dict,
#     "RAG_cols_with_samples": {},
#     "RAG_company_context": "",
#     "entire_table_context": {},
#     "template_df": {},
#     "result": [],
#     "error_message": "",
#     "iterations": 0,
#     "review_history_validator": []
# }

BUSINESS GLOSSARY FILLING - AGENTIC WORKFLOW

Project root: C:\Users\marcin.grzechowiak\Desktop\repos_learn\05_agent_dmo\business_glossary
Embedding model: text-embedding-3-small
LLM model: gpt-4o

⏳ Loading data from CSV files...
✅ Loaded sample dataset: 100 rows
✅ Loaded master glossary: 24 rows
✅ Loaded data stewards: 20 rows

Make sure these files exist:
  - C:\Users\marcin.grzechowiak\Desktop\repos_learn\05_agent_dmo\business_glossary\data\datasets\dataset_csv.csv
  - C:\Users\marcin.grzechowiak\Desktop\repos_learn\05_agent_dmo\business_glossary\data\master_business_glossary\master_business_glossary_csv.csv
  - C:\Users\marcin.grzechowiak\Desktop\repos_learn\05_agent_dmo\business_glossary\data\stewards_and_owners\data_stewards.csv


True

In [2]:
state = {
    "framework_def": config_datasets.get_framework_dict(),
    "source_original_table": sample_dict,
    "master_business_glossary": bg_dict,
    "master_data_owner": ds_dict,
    "RAG_cols_with_samples": {},
    "RAG_company_context": "",
    "entire_table_context": {},
    "template_df": {},
    "result": [],
    "error_message": "",
    "iterations": 0,
    "review_history_validator": []
}

In [3]:

import pandas as pd
from pathlib import Path
from langchain_openai import ChatOpenAI

# Internal Imports
from src.state import TemplateOutput, ValidationResult
from rag.config_rag import RAGConfig
from rag.retriever_formatting import PrepareRetrieval
from utils.helpers import template_enricher
# from config_paths import ConfigPaths
from configs.config_agent import ConfigAgents
from src.prompts import GENERATOR_PROMPT, VALIDATOR_PROMPT
from configs.config_datasets import DatasetColumnMappings

# --- LLM Setup ---
# config = ConfigPaths()
cfg_agent = ConfigAgents()
gpt_model = cfg_agent.llm_model

cfg_dataset = DatasetColumnMappings()

llm = ChatOpenAI(model=gpt_model, temperature=0)
structured_llm = llm.with_structured_output(TemplateOutput)
critic_llm = llm.with_structured_output(ValidationResult)

In [ ]:
#### NODE 1

In [4]:
"""Prepare the Business Glossary template skeleton from the sampled dataset."""
print("⏳ Fetching the template...")

cfg_dataset = DatasetColumnMappings()

original_sample_dict = state["source_original_table"]

df_template = cfg_dataset.build_template(original_sample_dict)

{"template_df": df_template.to_dict(orient="list")}
df_template

⏳ Fetching the template...


,bucket_name,dataset_name,table_name,column_name,sample_values,business_domain_name,business_sub_domain_name,business_name,column_description,attribute_rationale,attribute_rule,data_owner_name,data_owner_email
0,gs_bucket,client,client_account,account_id,"[AC100000, AC100001, AC100002]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
1,gs_bucket,client,client_account,client_name,"[Morgan Lopez, Riley Garcia, Morgan Anderson]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
2,gs_bucket,client,client_account,street_nm,"[1930 Oak Ave, 7099 Oak Blvd, 1412 Pine Rd]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
3,gs_bucket,client,client_account,acct_open_date,"[2015-01-13, 2017-05-02, 2018-03-28]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
4,gs_bucket,client,client_account,prod_code,"[PC-C, EQTY-7, PC-C]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
5,gs_bucket,client,client_account,risk_profile,"[Medium, High, Medium]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
6,gs_bucket,client,client_account,bal_local,"[1727237,59, 1959629,33, 1077348,73]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
7,gs_bucket,client,client_account,bal_usd,"[1969569,02, 2443069,89, 1137680,26]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
8,gs_bucket,client,client_account,cust_segment,"[SME, SME, SME]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
9,gs_bucket,client,client_account,ref_code,"[AGT0024, AGT0006, AGT0031]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>


In [5]:
from typing import Type
import pandas as pd
from pydantic import BaseModel
from src.state import ColumnDefInput   # ← fixed import

# optional: store schema in variable
column_def_schema = ColumnDefInput


def check_columns_with_pydantic(df: pd.DataFrame, schema: Type[BaseModel]) -> bool:
    """
    Raises ValueError if DataFrame columns are not exactly the same set
    as the Pydantic model fields (missing or extra columns).
    """
    expected = set(schema.model_fields.keys())
    actual = set(df.columns)

    missing = sorted(expected - actual)
    extra = sorted(actual - expected)

    if missing or extra:
        raise ValueError(
            f"DataFrame columns do not match schema {schema.__name__}.\n"
            f"Missing: {missing}\n"
            f"Extra:   {extra}"
        )
    return True

# usage
from src.state import ColumnDefInput
column_def_schema = ColumnDefInput
check_columns_with_pydantic(df_template, column_def_schema)


True

In [ ]:
### NODE 2

In [6]:
"""Enrich the template from the Master Business Glossary."""
print("\n⏳ Cross-checking with Master Business Glossary...")

framework_def = state.get("framework_def")
# df_template = pd.DataFrame(state.get("template_df"))
df_bg_glossary = pd.DataFrame(state.get("master_business_glossary"))

fill_cols = framework_def['search_with_RAG'] + framework_def['additional_col']

df_template_updated = template_enricher(
    template_df=df_template,
    enrich_df=df_bg_glossary,
    join_keys=["bucket_name", "dataset_name", 'table_name', 'column_name'],
    fill_cols=fill_cols,
)

{"template_df": df_template_updated.to_dict(orient='list')}
df_template_updated


⏳ Cross-checking with Master Business Glossary...


,bucket_name,dataset_name,table_name,column_name,sample_values,business_domain_name,business_sub_domain_name,business_name,column_description,attribute_rationale,attribute_rule,data_owner_name,data_owner_email
0,gs_bucket,client,client_account,account_id,ACC123456789,Client & Account Management,Account Master Data,Client Account Identifier,Unique identifier for a client account within ...,Enables unique identification and traceability...,Mandatory; unique; immutable once assigned,<ds_master>,<ds_master>
1,gs_bucket,client,client_account,client_name,ABC Holdings Ltd,Client & Account Management,Client Identity,Client Legal Name,Full legal name of the client as captured duri...,"Required for legal identification, contracts, ...",Mandatory; must match onboarding legal documen...,<ds_master>,<ds_master>
2,gs_bucket,client,client_account,street_nm,12 Baker Street,Client & Account Management,Client Address,Client Street Address,Primary mailing address field from onboarding;...,"Supports correspondence, jurisdiction checks, ...",Optional; free text,<ds_master>,<ds_master>
3,gs_bucket,client,client_account,acct_open_date,15.03.2022,Client & Account Management,Account Lifecycle,Account Opening Date,Date when the account was officially opened in...,"Determines account age, lifecycle stage, and r...",Mandatory; must be <= current date,<ds_master>,<ds_master>
4,gs_bucket,client,client_account,prod_code,"[PC-C, EQTY-7, PC-C]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
5,gs_bucket,client,client_account,risk_profile,"[Medium, High, Medium]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
6,gs_bucket,client,client_account,bal_local,125000.50,Finance,Account Balances,Local Currency Account Balance,Account balance in the client's local reportin...,"Used for client reporting, interest calculatio...",Numeric; may be negative,<ds_master>,<ds_master>
7,gs_bucket,client,client_account,bal_usd,137500.75,Finance,Account Balances,USD Equivalent Account Balance,Account balance converted to USD using interna...,Enables consolidated financial and risk report...,Derived using approved FX rates,<ds_master>,<ds_master>
8,gs_bucket,client,client_account,cust_segment,"[SME, SME, SME]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>
9,gs_bucket,client,client_account,ref_code,"[AGT0024, AGT0006, AGT0031]",<agent>,<agent>,<agent>,<agent>,<agent>,<agent>,<ds_master>,<ds_master>


In [ ]:
### NODE 3 ####

In [8]:
print("⏳ Cross-checking with Master Data Owner/Steward File...")

######################## PART 1/2 ########################
# Perform the enrichment using Master Data Owner/Steward file
framework_def = state.get("framework_def")
df_template = df_template_updated.copy() #pd.DataFrame(state.get("template_df"))
df_do_master = pd.DataFrame(state.get("master_data_owner"))

# Enrichment
fill_cols = framework_def['search_with_data_steward_file']
df_template_updated = template_enricher(
    template_df=df_template,
    enrich_df=df_do_master,
    join_keys=["bucket_name", "dataset_name", 'table_name'], # stewards are joined on table level
    fill_cols=fill_cols,
)

full_dict = df_template_updated.to_dict(orient='list')

print("✅ Master files has been reviewed! Following structure will be sent to an Agent to generate missing fields:")
print(df_template_updated.head(5))

⏳ Cross-checking with Master Data Owner/Steward File...
✅ Master files has been reviewed! Following structure will be sent to an Agent to generate missing fields:
  bucket_name dataset_name      table_name     column_name  \
0   gs_bucket       client  client_account      account_id   
1   gs_bucket       client  client_account     client_name   
2   gs_bucket       client  client_account       street_nm   
3   gs_bucket       client  client_account  acct_open_date   
4   gs_bucket       client  client_account       prod_code   

          sample_values         business_domain_name business_sub_domain_name  \
0          ACC123456789  Client & Account Management      Account Master Data   
1      ABC Holdings Ltd  Client & Account Management          Client Identity   
2       12 Baker Street  Client & Account Management           Client Address   
3            15.03.2022  Client & Account Management        Account Lifecycle   
4  [PC-C, EQTY-7, PC-C]                      <agent>       

In [ ]:
df_template_updated

In [9]:
######################## PART 2/2 ########################
### When full template is ready, filter only those rows which need to be filled in by RAG
# (keep only rows with placeholders in RAG columns) - this will be used for retrieval and building
# the context for the generator

# Restrict RAG to only columns that still contain placeholders (extract rows which contains tag <...>)
pattern = r"<.*?>"
df_missing = df_template_updated[
    df_template_updated.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)
]

## Below object will be used to perform RAG searches (column name + sample values,
# e.g. Account_number: [12345, 67890, 111213])
dict_for_RAG_search = dict(zip(df_missing["column_name"], df_missing["sample_values"]))



In [ ]:
dict_for_RAG_search

In [ ]:
## Node 4

In [10]:
col_samples = dict_for_RAG_search.copy() #state.get("RAG_cols_with_samples")

# Initialize RAG (assuming paths are relative to root where script is run)
cfg = RAGConfig(project_root=Path.cwd())
prep = PrepareRetrieval(cfg)

results = prep.retrieve_for_all_columns(col_samples)
company_context_prompt = prep.build_prompt_and_format(results)

{"RAG_company_context": company_context_prompt}

=== RAG ===
Loading vector database from C:\Users\marcin.grzechowiak\Desktop\repos_learn\05_agent_dmo\business_glossary\vector_dbs\chroma_db...
✅ Loaded database with 124 chunks
Retrieving 1 chunk(s) for query: 'Find relevant information for the following variable: Column Name: 'prod_code'. Example Values: PC-C, EQTY-7, PC-C. '
✅ Retrieved 1 chunk(s)
Retrieving 1 chunk(s) for query: 'Find relevant information for the following variable: Column Name: 'risk_profile'. Example Values: Medium, High, Medium. '
✅ Retrieved 1 chunk(s)
Retrieving 1 chunk(s) for query: 'Find relevant information for the following variable: Column Name: 'cust_segment'. Example Values: SME, SME, SME. '
✅ Retrieved 1 chunk(s)
Retrieving 1 chunk(s) for query: 'Find relevant information for the following variable: Column Name: 'ref_code'. Example Values: AGT0024, AGT0006, AGT0031. '
✅ Retrieved 1 chunk(s)
Retrieving 1 chunk(s) for query: 'Find relevant information for the following variable: Column Name: 'tx_code'. E

{'RAG_company_context': '{\n  "columns": [\n    {\n      "column_name": "prod_code",\n      "hits": [\n        {\n          "hit_id": "prod_code#1",\n          "text": "Key information:  If there is a name such as prod_code it Uses business product shortcodes used by Sales and Ops. Look for mapping in Sales Product Matrix.",\n          "source": "C:\\\\Users\\\\marcin.grzechowiak\\\\Desktop\\\\repos_learn\\\\05_agent_dmo\\\\business_glossary\\\\data\\\\docs\\\\aurelia_data_info.docx"\n        }\n      ]\n    },\n    {\n      "column_name": "risk_profile",\n      "hits": [\n        {\n          "hit_id": "risk_profile#1",\n          "text": "gs_bucket,risk,risk_monitor,aml_alert_level,Risk & Compliance,AML Monitoring,AML Risk Alert Level,AML monitoring alert level (None/Low/Watch/High).,High,Supports financial crime monitoring and escalation workflows.,Enumerated list,Marcus Thorne,marcus.thorne@aureliacap.com",\n          "source": "C:\\\\Users\\\\marcin.grzechowiak\\\\Desktop\\\\repos

In [ ]:
## Node 5

In [ ]:
company_context_prompt

In [ ]:
print(f"--- GENERATOR: Filling the template (Attempt {state['iterations'] + 1}) ---")

# Bring context from state
full_table_context = full_dict.copy() #json.dumps(state.get('entire_table_context'), indent=2) # Convert dict to JSON on the fly so it's better formatted when supplying to the Agent
rag_company_context = company_context_prompt #(state.get('RAG_company_context', "No additional context provided."))

# Prepare critic feedback if available
critic_feedback = ""
if state['error_message'] != "none":
    critic_feedback = f"\n\nCRITIC FEEDBACK FROM PREVIOUS ATTEMPT:\n{state['error_message']}\nPlease fix these issues."

# Format the prompt using LangChain's template
formatted_messages = GENERATOR_PROMPT.format_messages(
    full_table_context=full_table_context,
    rag_company_context=rag_company_context,
    critic_feedback=critic_feedback
    )

response = structured_llm.invoke(formatted_messages)
# return {
#     "result": response.columns,
#     "iterations": state['iterations'] + 1

In [11]:
"""5. Define the Generator Agent logic"""

print(f"--- GENERATOR: Filling the template (Attempt {state['iterations'] + 1}) ---")

# Bring context from state
full_table_context = full_dict.copy()# json.dumps(state.get('entire_table_context'), indent=2) # Convert dict to JSON on the fly so it's better formatted when supplying to the Agent
rag_company_context = company_context_prompt #(state.get('RAG_company_context', "No additional context provided."))

# Prepare critic feedback if available
critic_feedback = ""
if state['error_message'] != "none":
    critic_feedback = f"\n\nCRITIC FEEDBACK FROM PREVIOUS ATTEMPT:\n{state['error_message']}\nPlease fix these issues."

# Format the prompt using LangChain's template
formatted_messages = GENERATOR_PROMPT.format_messages(
    full_table_context=full_table_context,
    rag_company_context=rag_company_context,
    critic_feedback=critic_feedback
    )

response = structured_llm.invoke(formatted_messages)
# return {
#     "result": response,
#     "iterations": state['iterations'] + 1
# }

--- GENERATOR: Filling the template (Attempt 1) ---


In [16]:
response.table_summary

src.state.TemplateOutput

In [ ]:
formatted_result = [{"row_number": i + 1, **c.model_dump()}
                    for i, c in enumerate(response.columns)]

In [ ]:
## Node 6

In [ ]:
print("--- CRITIC: Reviewing... ---")

# Calculate counts to check for data loss
input_table = full_dict.copy() #state.get('entire_table_context', {})
expected_count = len(input_table.get('Column Name', []))
actual_count = len(response.columns)

current_work = "\n".join([
    f"Column: {c.column_name}\n"
    f"Proposed Description: {c.column_description}\n"
    f"Source Used: {c.extra__add_source_explained}\n"
    f"Evidence/Logic: {c.extra__add_citation_of_the_hit}\n"
    "---"
    for c in response.columns
])

# Bring context from state
rag_company_context = state.get('RAG_company_context', "No additional context provided.")
full_table_context = input_table #state.get('entire_table_context', {})

# Format the prompt using LangChain's template
formatted_messages = VALIDATOR_PROMPT.format_messages(
    rag_company_context=rag_company_context,
    full_table_context=full_table_context,
    current_work=current_work,
    expected_count=expected_count,
    actual_count=actual_count
)

review = critic_llm.invoke(formatted_messages)

# if false (i.e. not valid)
if not review.is_valid:
    print(f"--- CRITIC FEEDBACK: {review.feedback} ---")
    print(f"error_message",{review.feedback})
    # print("review_history_validator", {[f"Step {state['iterations']} Critic: {review.feedback}"]

# print("error_message", "none")
    # "review_history_validator": [f"Critique (Passed): {review.feedback}"])

In [ ]:
review

In [ ]:
pd.DataFrame(full_dict)

In [ ]:
print(company_context_prompt)